In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cache_sizes = np.arange(1, 100)
n_pages = 100
n_access = 10000

In [ ]:
cache_sizes

# Blub

In [ ]:
def find_next_usage(page, accesses):
    counter = 0
    for page_next in accesses:
        if page_next == page:
            break
        else:
            counter += 1
    else:
        counter = np.inf
    return counter

In [ ]:
class Cache:
    def __init__(self, size):
        self.size = size
    def replace(self, x):
        pass
    def contains(self, x):
        pass

class OptimalCache:
    def __init__(self, size):
        self.size = size
        self.cache = set([])
    
    name = "Optimal"
    
    def contains(self, x):
        return x in self.cache
    
    def __len__(self):
        return len(self.cache)
    
    def is_full(self):
        return len(self.cache) >= self.size
    
    def add(self, page):
        assert len(self.cache) < self.size
        self.cache.add(page)
    
    def replace(self, page, future_workload):
        future_usage = [(p, find_next_usage(p, future_workload)) for p in self.cache]
        replace_page = max(future_usage, key=lambda x: x[1])
        self.cache.remove(replace_page[0])
        self.add(page)

class FIFOCache:
    def __init__(self, size):
        self.size = size
        self.cache = []
    
    name = "FIFO"
    
    def contains(self, x):
        return x in self.cache

    def __len__(self):
        return len(self.cache)
    
    def is_full(self):
        return len(self.cache) >= self.size

    def add(self, page):
        assert len(self.cache) < self.size
        self.cache.insert(0, page)
    
    def replace(self, page, *args):
        self.cache.pop()
        self.cache.insert(0, page)

class RandomCache:
    def __init__(self, size):
        self.size = size
        self.cache = []
    
    name = "Random"
    
    def contains(self, x):
        return x in self.cache

    def __len__(self):
        return len(self.cache)
    
    def is_full(self):
        return len(self.cache) >= self.size

    def add(self, page):
        assert len(self.cache) < self.size
        self.cache.append(page)
    
    def replace(self, page, *args):
        self.cache.pop(np.random.randint(self.size))
        self.add(page)

class LRUCache:
    def __init__(self, size):
        self.size = size
        self.cache = []

    name = "LRU"
    
    def contains(self, page):
        if page in self.cache:
            self.cache.remove(page)
            self.add(page)
            return True
        else:
            return False

    def __len__(self):
        return len(self.cache)
    
    def is_full(self):
        return len(self.cache) >= self.size

    def add(self, page):
        assert len(self.cache) < self.size
        assert page not in self.cache
        self.cache.insert(0, page)
    
    def replace(self, page, *args):
        self.cache.pop()
        self.add(page)

In [ ]:
def calc_hit_rate(cache, workload):
    n_hits = 0
    n_miss = 0
    for idx, page in enumerate(workload):
        if cache.contains(page):
            n_hits += 1
        else:
            n_miss += 1
            if cache.is_full():
                cache.replace(page, workload[idx + 1:])
            else:
                cache.add(page)
    return n_hits / (n_hits + n_miss)

In [ ]:
def get_random_workload(n_pages, n_access):
    return np.random.randint(0, n_pages, n_access)

def get_80_20_workload(n_pages, n_access):
    n_hot = int(0.2 * n_pages)
    n_cold = n_pages - n_hot
    #
    pages = np.arange(n_pages)
    np.random.shuffle(pages)
    hot_pages = pages[:n_hot]
    cold_pages = pages[n_hot:]
    #
    n_access_hot = int(n_access * 0.8)
    n_access_cold = n_access - n_access_hot
    #
    workload_hot = np.random.choice(hot_pages, size=n_access_hot)
    workload_cold = np.random.choice(cold_pages, size=n_access_cold)
    workload = np.concatenate([workload_hot, workload_cold])
    for _ in range(10):
        np.random.shuffle(workload)
    return workload

def get_sequential_workload(n_pages, n_access):
    n_repetitions = int(n_access / n_pages)
    return np.tile(np.arange(n_pages), n_repetitions)

In [ ]:
CACHES = [LRUCache, RandomCache, FIFOCache, OptimalCache]    

# General Settings

In [ ]:
n_pages = 100
n_access = 10000
#
CACHES = [LRUCache, RandomCache, FIFOCache, OptimalCache]    

# Random Workload

In [ ]:
workload = get_random_workload(n_pages, n_access)
#
results = {}
for CACHE in CACHES:
    hit_rates = []
    for cache_size in range(1, n_pages + 1, 1):
        cache = CACHE(cache_size)
        hit_rate = calc_hit_rate(cache, workload)
        hit_rates.append(hit_rate)
        print(cache.name, cache_size, hit_rate)
    results[CACHE.name] = hit_rates
results_rand = results

In [ ]:
plt.figure(figsize=(10,10))
xx = list(range(1, n_pages + 1, 1))
plt.plot(xx, results["LRU"], label="LRU", color="blue")
plt.plot(xx, results["Random"], label="Random", color="green")
plt.plot(xx, results["FIFO"], label="FIFO", color="red")
plt.plot(xx, results["Optimal"], label="Optimal", color="orange")
plt.legend()

# 80 20

In [ ]:
workload = get_80_20_workload(n_pages, n_access)
#
results = {}
for CACHE in CACHES:
    hit_rates = []
    for cache_size in range(1, n_pages + 1, 1):
        cache = CACHE(cache_size)
        hit_rate = calc_hit_rate(cache, workload)
        hit_rates.append(hit_rate)
        print(cache.name, cache_size, hit_rate)
    results[CACHE.name] = hit_rates
results_8020 = results

In [ ]:
plt.figure(figsize=(10,10))
xx = list(range(1, n_pages + 1, 1))
plt.plot(xx, results["LRU"], label="LRU", color="blue")
plt.plot(xx, results["Random"], label="Random", color="green")
plt.plot(xx, results["FIFO"], label="FIFO", color="red")
plt.plot(xx, results["Optimal"], label="Optimal", color="orange")
plt.legend()

# Sequential

In [ ]:
n_pages = 50
cache_sizes = range(1, 100 + 1, 1)
workload = get_sequential_workload(n_pages, n_access)
#
results = {}
for CACHE in CACHES:
    hit_rates = []
    for cache_size in cache_sizes:
        cache = CACHE(cache_size)
        hit_rate = calc_hit_rate(cache, workload)
        hit_rates.append(hit_rate)
        print(cache.name, cache_size, hit_rate)
    results[CACHE.name] = hit_rates
results_seq = results

In [ ]:
plt.figure(figsize=(10,10))
xx = cache_sizes
#
plt.plot(xx, results["LRU"], label="LRU", color="blue")
plt.plot(xx, results["Random"], label="Random", color="green")
plt.plot(xx, results["FIFO"], label="FIFO", color="red")
plt.plot(xx, results["Optimal"], label="Optimal", color="orange")
plt.legend()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

#
xx = cache_sizes
# Random
axes[0].plot(xx, results_rand["LRU"], label="LRU", color="blue", linewidth="4", alpha=0.5)
axes[0].plot(xx, results_rand["Random"], label="Random", color="green", linewidth="4", alpha=0.7)
axes[0].plot(xx, results_rand["FIFO"], label="FIFO", color="red", linestyle=":")
axes[0].plot(xx, results_rand["Optimal"], label="Optimal", color="orange", linewidth="3")
axes[0].set_ylabel("hit rate")
axes[0].set_xlabel("Cache size / pages")
axes[0].set_title("Random Workload")
axes[0].legend()
# 80 : 20
axes[1].plot(xx, results_8020["LRU"], label="LRU", color="blue", linewidth="4", alpha=0.5)
axes[1].plot(xx, results_8020["Random"], label="Random", color="green", linewidth="4", alpha=0.5)
axes[1].plot(xx, results_8020["FIFO"], label="FIFO", color="red", linestyle=":")
axes[1].plot(xx, results_8020["Optimal"], label="Optimal", color="orange", linewidth="3")
axes[1].set_ylabel("hit rate")
axes[1].set_xlabel("Cache size / pages")
axes[1].set_title("80-20 Workload")
axes[1].legend()
# Sequential
axes[2].plot(xx, results_seq["LRU"], label="LRU", color="blue", linewidth="4", alpha=0.5)
axes[2].plot(xx, results_seq["Random"], label="Random", color="green", linewidth="4", alpha=0.5)
axes[2].plot(xx, results_seq["FIFO"], label="FIFO", color="red", linestyle=":")
axes[2].plot(xx, results_seq["Optimal"], label="Optimal", color="orange", linewidth="3")
axes[2].set_xlabel("Cache size / pages")
axes[2].set_title("Sequential Workload")
axes[2].set_ylabel("hit rate")
axes[2].legend()
#plt.legend()
#plt.suptitle("Workloads")
fig.tight_layout()
plt.savefig("policies_wl.pdf")
